# Initial settings & Reading data

## Install Packages

In [89]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

import requests
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go

import geopandas as gpd
from shapely.geometry import Point

from IPython.display import HTML

## Display Options

In [91]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)

In [92]:
#pd.options.display.float_format = '{:,}'.format

In [93]:
pd.options.display.float_format = "{:,.2f}".format

In [94]:
#pd.set_option("styler.format.thousands", ",")

In [95]:
#df['Qty'] = df['Qty'].apply('{:,}'.format)
#df['Total Revenue'] = df['Total Revenue'].apply('{:,}'.format)

## General Tickets

In [97]:
general_tickets = pd.read_excel('dataset/general_tickets.xlsx')

In [98]:
df = pd.read_csv('df.csv')

In [99]:
df.head(2)

,id,order_item_id,Ticket,Type,Price,Coupon,Qty,Total Revenue,Ticket Type,Outcode,Incode,Postcode,Latitude,Longitude,pcd_sect,pcd_dist,pcd_area,Postcode Area,pop_area,pop_dist,pop_sect
0,78358,74791,PSF - St Albans City v Cambridge United - Next...,Next Gen 18-23,6.00,NaN,1,6.00,General,CB23,8TL,CB23 8TL,52.25,0.03,CB23 8,CB23,CB,Cambridge,"463,870.00","36,070.00","5,048.00"
1,78395,74801,PSF - St Albans City v Cambridge United - Conc...,Concession,8.00,NaN,1,8.00,General,W12,9RH,W12 9RH,51.50,-0.25,W12 9,W12,W,London W,"531,439.00","49,515.00","13,159.00"


In [100]:
df['Qty'].sample(n=1000, random_state=1, replace = True)

235      1
5192     3
905      1
10955    1
7813     2
        ..
9992     2
4533     2
7698     1
10261    2
3139     1
Name: Qty, Length: 1000, dtype: int64

## General Tickets

In [102]:
general_tickets = pd.read_excel('dataset/general_tickets.xlsx')

In [103]:
general_tickets['Price'].sum()

126330.28999999998

In [104]:
general_tickets['Ticket Type'] = 'General'

## Season Tickets

In [106]:
season_tickets = pd.read_excel('dataset/season_tickets.xlsx')

In [107]:
season_tickets['Ticket Type'] = 'Season'

## Tickets Concatinated

In [109]:
tickets = pd.concat([general_tickets, season_tickets], axis=0)

#tickets.info()

In [110]:
# Drop column name Channel - only one value

tickets.drop('Channel', axis=1, inplace = True)

In [111]:
tickets.rename({'Price':'Total Revenue'}, axis = 1, inplace = True)
tickets['Price'] = tickets['Total Revenue']/tickets['Qty']

In [112]:
# Amend postcode names to the right terminology

tickets.rename({'Postcode':'Outcode', 'Column1':'Incode'}, axis = 1, inplace = True)

In [113]:
tickets['Postcode'] = tickets['Outcode'].astype(str) + " " + tickets['Incode'].astype(str)
#tickets['Sector'] = tickets['Outcode'].astype(str) + " " + tickets['Incode'].astype(str).str[0]

tickets = tickets[['id', 'order_item_id', 'Ticket', 'Type', 'Price', 'Coupon', 'Qty', 'Total Revenue', 'Ticket Type', 'Outcode', 'Incode',  #'Sector', 
         'Postcode']]

In [114]:
tickets[tickets['Outcode'].isna() == True].head(2) # 660 rows have no postcode, but Postcode seem as "nan nan" because of the conversion

,id,order_item_id,Ticket,Type,Price,Coupon,Qty,Total Revenue,Ticket Type,Outcode,Incode,Postcode
81,106917,101533,PSF - St Albans City v Cambridge United - Adult,Adult,0.00,scout,4,0.00,General,NaN,NaN,nan nan
82,106923,101539,PSF - St Albans City v Maidenhead United - Adult,Adult,0.00,scout,1,0.00,General,NaN,NaN,nan nan


In [115]:
11719 + 625 # Total row nr is 12344 & correct

12344

In [116]:
###################################################################################################################
#
tickets.replace('nan nan', np.nan, inplace=True)
tickets.replace('nan n', np.nan, inplace=True)
#
#tickets[tickets['Outcode'].isna() == True].head()

In [117]:
tickets.head(3)

,id,order_item_id,Ticket,Type,Price,Coupon,Qty,Total Revenue,Ticket Type,Outcode,Incode,Postcode
0,78358,74791,PSF - St Albans City v Cambridge United - Next...,Next Gen 18-23,6.00,NaN,1,6.00,General,CB23,8TL,CB23 8TL
1,78395,74801,PSF - St Albans City v Cambridge United - Conc...,Concession,8.00,NaN,1,8.00,General,W12,9RH,W12 9RH
2,78397,74803,PSF - St Albans City v Cambridge United - Adult,Adult,10.00,NaN,1,10.00,General,CO10,8RH,CO10 8RH


In [118]:
tickets['Postcode'].isna().sum()

660

---
## Latitude and Altitude 

In [120]:
# Office for National Statistics

# ONS Postcode Directory (August 2024) for the UK 
# Obtained from: https://ons.maps.arcgis.com/home/search.html?restrict=true&sortField=relevance&sortOrder=desc&searchTerm=tags%3AONS+Postcode+Directory#content)

ukpostcodes = pd.read_csv('dataset/ONSPD_AUG_2024_UK.csv', usecols = ['pcd2', 'lat', 'long'])

In [121]:
ukpostcodes.head()

,pcd2,lat,long
0,AB1 0AA,57.10,-2.24
1,AB1 0AB,57.10,-2.25
2,AB1 0AD,57.10,-2.25
3,AB1 0AE,57.08,-2.26
4,AB1 0AF,57.10,-2.26


In [122]:
ukpostcodes['pcd2'] = ukpostcodes['pcd2'].str.replace('\s{2,}', ' ', regex = True)

In [123]:
ukpostcodes['pcd2'][0]

'AB1 0AA'

In [124]:
ukpostcodes.rename({
    'pcd2':'Postcode', 
    "lat": "Latitude", 
    "long":"Longitude"}, axis = 1, inplace = True)

In [125]:
ukpostcodes.head()

,Postcode,Latitude,Longitude
0,AB1 0AA,57.10,-2.24
1,AB1 0AB,57.10,-2.25
2,AB1 0AD,57.10,-2.25
3,AB1 0AE,57.08,-2.26
4,AB1 0AF,57.10,-2.26


In [126]:
# Spot checking if the latitude and longitude matches the postcode

#ukpostcodes[ukpostcodes['pcd2'].str.startswith("AL1 3UR") == True]

#ukpostcodes.iloc[[2200000]]

---
## Postcode, Sector, Area

In [128]:
# Postcode ~ Postcode Sector ~ Postcode District to Postcode Area (August 2022) to Output Area (2021) Lookup in EW

# https://geoportal.statistics.gov.uk/datasets/ons::postcode-to-postcode-sector-to-postcode-district-to-postcode-area-august-2022-to-output-area-2021-lookup-in-ew/about

# pcd2: Postcode
# pcd_dist: District
# pcd_area: Area

postcode_areas = pd.read_csv('dataset/pcd_pcds_pcdd_pcda_oa21_aug_22__ew_lu.csv', usecols = ['pcd2', 'pcd_dist', 'pcd_area', 'pcd_sect'])

In [129]:
postcode_areas.head()

,pcd2,pcd_sect,pcd_dist,pcd_area
0,AL1 1AA,AL1 1,AL1,AL
1,AL1 1AB,AL1 1,AL1,AL
2,AL1 1AD,AL1 1,AL1,AL
3,AL1 1AE,AL1 1,AL1,AL
4,AL1 1AF,AL1 1,AL1,AL


In [130]:
#Two space to one space

In [131]:
postcode_areas['pcd_area'] = postcode_areas['pcd_area'].str.split('\d', expand=True)[0]

In [132]:
postcode_areas['pcd2'] = postcode_areas['pcd2'].str.replace('\s{2,}', ' ', regex = True)

In [133]:
# Sector has an issue with areas with double digits

postcode_areas[postcode_areas['pcd_dist'] == 'AL10'].head()

,pcd2,pcd_sect,pcd_dist,pcd_area
2017,AL10 0AA,AL100,AL10,AL
2018,AL10 0AB,AL100,AL10,AL
2019,AL10 0AD,AL100,AL10,AL
2020,AL10 0AE,AL100,AL10,AL
2021,AL10 0AF,AL100,AL10,AL


In [134]:
def extract_sect(pcd):
    if len(pcd) >= 4 and pcd[3] == ' ':
        return pcd[:5]  # Get the first 5 characters (e.g., "AL1 1")
    elif len(pcd) >= 5 and pcd[4] == ' ':
        return pcd[:6]  # Get the first 6 characters (e.g., "AL10 0")
    else:
        return None  # Return None if it doesn't match the expected format


In [135]:
#Fixing sector
postcode_areas['pcd_sect'] = postcode_areas['pcd2'].apply(extract_sect)

In [136]:
postcode_areas[postcode_areas['pcd_dist'] == 'AL10'].head()

,pcd2,pcd_sect,pcd_dist,pcd_area
2017,AL10 0AA,AL10 0,AL10,AL
2018,AL10 0AB,AL10 0,AL10,AL
2019,AL10 0AD,AL10 0,AL10,AL
2020,AL10 0AE,AL10 0,AL10,AL
2021,AL10 0AF,AL10 0,AL10,AL


In [137]:
postcode_areas.head()

,pcd2,pcd_sect,pcd_dist,pcd_area
0,AL1 1AA,AL1 1,AL1,AL
1,AL1 1AB,AL1 1,AL1,AL
2,AL1 1AD,AL1 1,AL1,AL
3,AL1 1AE,AL1 1,AL1,AL
4,AL1 1AF,AL1 1,AL1,AL


In [138]:
# Mental check: Does the *Total Nr of Postcodes per Postcode Area* match with the *Total* column in postcode_area_names data

#postcode_areas[['pcd2', 'pcd_area']].groupby('pcd_area').count().head()

---
## Postcode Area Names

In [140]:
postcode_area_names = pd.read_csv('dataset/Postcode Area Names (ONSPD User Guide Aug 2024).csv')#, usecols = ['Postcode Area', 'Postcode Area Name', 'Region','Status'])

In [141]:
postcode_area_names.head()

,Postcode Area,Post code Area Code,Postcode Districts,Post code Sectors,Live,Terminated,Total
0,Aberdeen,AB,40,180,"17,372","21,869","39,241"
1,St Albans,AL,10,39,"7,791","3,678","11,469"
2,Birmingham,B,79,268,"41,907","20,621","62,528"
3,Bath,BA,19,81,"15,490","5,017","20,507"
4,Blackburn,BB,15,79,"13,532","5,741","19,273"


In [142]:
postcode_area_names.rename(columns = {'Postcode Area ':'Postcode Area'},inplace = True)

In [143]:
postcode_area_names['Post code Area Code'] = postcode_area_names['Post code Area Code'].str.strip(' ') #.str.contains(' ').sum()  All rows have space after the area code

In [144]:
#postcode_area_names.info()

## pcdf ~ Latitude Altitude & Postcode Areas & Area Names Merged

In [146]:
#postcode_areas.merge(
#    postcode_area_names, how = 'left', left_on ='pcd_area', right_on = 'Post code Area Code').head(2)

In [147]:
pcdf = ukpostcodes.merge(
    postcode_areas.merge(postcode_area_names, how = 'left', left_on ='pcd_area', right_on = 'Post code Area Code'),
    how = 'left', left_on ='Postcode', right_on = 'pcd2')

### It's ok to have NAs !!

In [149]:
pcdf.head(2)

,Postcode,Latitude,Longitude,pcd2,pcd_sect,pcd_dist,pcd_area,Postcode Area,Post code Area Code,Postcode Districts,Post code Sectors,Live,Terminated,Total
0,AB1 0AA,57.10,-2.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AB1 0AB,57.10,-2.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
#pcdf.drop(columns = 'pcd2').head()

In [151]:
postcode_areas[postcode_areas['pcd_area'] == 'AL'].head(3)

,pcd2,pcd_sect,pcd_dist,pcd_area
0,AL1 1AA,AL1 1,AL1,AL
1,AL1 1AB,AL1 1,AL1,AL
2,AL1 1AD,AL1 1,AL1,AL


In [152]:
# Columns Postcode and pcd2 are identical where pcd2 is not null

(pcdf[(pcdf['Postcode'] == pcdf['pcd2']) & (pcdf['pcd2'].isnull() == False)]['Postcode'] != 
 pcdf[(pcdf['Postcode'] == pcdf['pcd2']) & (pcdf['pcd2'].isnull() == False)]['pcd2']).sum()

0

In [153]:
pcdf.drop(pcdf[pcdf['pcd2'].isna() == True].index.tolist(), inplace = True)

In [154]:
pcdf.head(2)

,Postcode,Latitude,Longitude,pcd2,pcd_sect,pcd_dist,pcd_area,Postcode Area,Post code Area Code,Postcode Districts,Post code Sectors,Live,Terminated,Total
39241,AL1 1AA,51.75,-0.34,AL1 1AA,AL1 1,AL1,AL,St Albans,AL,10.00,39.00,"7,791","3,678","11,469"
39242,AL1 1AB,51.75,-0.30,AL1 1AB,AL1 1,AL1,AL,St Albans,AL,10.00,39.00,"7,791","3,678","11,469"


#### Rows where incode is null, makes 0.43% of the revenue. Therefore, I will drop these rows

In [156]:
# 0.4% has no outcode

pcdf[(pcdf['Outcode'].isna() == False) & (pcdf['Incode'].isna() == True)]['Total Revenue'].sum() / pcdf['Total Revenue'].sum()*100

KeyError: 'Outcode'

In [ ]:
pcdf.drop(index =  pcdf[(pcdf['Incode'].isna() == True) & (pcdf['Outcode'].isna() == False)].index.tolist(), inplace = True)

In [ ]:
pcdf[(pcdf['Incode'].isna() == True) & (pcdf['Outcode'].isna() == False)].head()#.index.tolist()

## Merged tickets ~ Tickets and Postcodes

In [ ]:
tickets_pcd = tickets.merge(pcdf[['Postcode', 'Latitude', 'Longitude', 'pcd_sect', 'pcd_dist', 'pcd_area', 'Postcode Area']], how = 'left', left_on = 'Postcode', right_on = 'Postcode')

tickets_pcd.head(2)

In [ ]:
# Revenue belonging to below null rows negligible (0.39%)

tickets_pcd[
(tickets_pcd['Latitude'].isna() == True) | 
(tickets_pcd['Longitude'].isna() == True) | 
(tickets_pcd['pcd_sect'].isna() == True) | 
(tickets_pcd['pcd_dist'].isna() == True) | 
(tickets_pcd['pcd_area'].isna() == True)
]['Total Revenue'].sum() / tickets_pcd['Total Revenue'].sum()

In [ ]:
tickets_pcd.drop(index = 
       tickets_pcd[(tickets_pcd['Latitude'].isna() == True) | 
        (tickets_pcd['Longitude'].isna() == True) | 
        (tickets_pcd['pcd_sect'].isna() == True) | 
        (tickets_pcd['pcd_dist'].isna() == True) | 
        (tickets_pcd['pcd_area'].isna() == True)].index.tolist(), inplace = True)

---
## Population

In [ ]:
# Postcode resident and household estimates, England and Wales: Census 2021

# https://www.nomisweb.co.uk/sources/census_2021_pc
# Table 1: All postcodes (1.3m records)

population = pd.read_csv('dataset/Postcode_Estimates_Table_2021.csv', usecols = ['Postcode', 'Count'])

In [ ]:
#population.rename(column = {'Count' = 'Postcode Count'})

In [ ]:
population.head()

In [ ]:
# Checking if the dataset matches the population of England and Wales for 2021

#population['Count'].sum()
#
#UKpop2021 =  59642000 #England and Wales
#
#error = (UKpop2021 - population['Count'].sum())/UKpop2021*100
#
#error # 0.07 %

In [ ]:
# I don't want any postcodes to reappear in the dataset so I will group them and sum the Count column

population = population.groupby('Postcode', as_index = False).sum()

In [ ]:
population.head()

In [ ]:
population = population.merge(pcdf[['Postcode',
                       #'Latitude',
                       #'Longitude',
                       #'pcd2',
                       'pcd_sect',
                       'pcd_dist',
                       'pcd_area',
                       'Postcode Area',
                       #'Post code Area Code',
                       #'Postcode Districts',
                       #'Post code Sectors',
                       #'Live ',
                       #'Terminated ',
                       #'Total'
                      ]],
                how = 'left',
                left_on = 'Postcode',
                right_on = 'Postcode')

In [ ]:
population.head()

In [ ]:
popdf = population.merge(
    population.groupby('pcd_area', as_index = False).sum('Count'),
    how = 'left',
    left_on = 'pcd_area',
    right_on = 'pcd_area',
    suffixes = ('', '_area')
).merge(
    population.groupby('pcd_dist', as_index = False).sum('Count'),
    how = 'left',
    left_on = 'pcd_dist',
    right_on = 'pcd_dist',
    suffixes = ('', '_dist')
).merge(
    population.groupby('pcd_sect', as_index = False).sum('Count'),
    how = 'left',
    left_on = 'pcd_sect',
    right_on = 'pcd_sect',
    suffixes = ('', '_sect'))#.iloc[[40,1662,990,2328,385477, 646946]]

In [ ]:
popdf.rename(
    columns = {
        'Count_area' : 'pop_area',
        'Count_dist' : 'pop_dist',
        'Count_sect' : 'pop_sect'
    },
    inplace = True
)

In [ ]:
popdf.iloc[[40,1662,990,2328,385477, 646946]]

In [ ]:
#popdf[popdf['pcd_area'] == 'WC'].head() # 30,000 WC
# popdf[popdf['pcd_area'] == 'NW'].head() # 574,852 NW

In [ ]:
#Checking if the merge is correct

In [ ]:
#popdf[popdf['Postcode'].str.startswith('LU')].head()

In [ ]:
#pop_by_area[pop_by_area['pcd_area'].str.startswith('LU')].head()

In [ ]:
#pop_by_district[pop_by_district['pcd_dist'].str.startswith('LU1')].head()

In [ ]:
#pop_by_sect[pop_by_sect['pcd_sect'].str.startswith('AL1 1')].head()

In [ ]:
#pop_by_sect = population[['Postcode Area', 'pcd_sect', 'Count']].groupby('pcd_sect', as_index = False).agg({
#    'Postcode Area': 'first', 
#    'Count': 'sum'})

In [ ]:
#pop_by_sect.head()

---
## Merged df ~ population

In [ ]:
df = tickets_pcd.merge(popdf[
         ['Postcode',
         'pop_area',
         'pop_dist',
         'pop_sect']],
        how = 'left',
        left_on = 'Postcode',
        right_on = 'Postcode')

---
## Write df to csv

In [ ]:
df.head(2)

In [ ]:
df.to_csv('df.csv', index=False)

In [ ]:
#df.head(2)

# Grouped Data

## Group by Area

In [ ]:
df['Type'].unique()

In [ ]:
df_by_area = df.groupby(['pcd_area', 'Ticket Type'], as_index=False).agg({ #[df['Price'] != 0]
    'Postcode Area': 'first', 
    'Qty': 'sum',
    'Total Revenue': 'sum', 
    'pop_area': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
}).sort_values(by = 'Total Revenue', ascending =False)

df_by_area.head(2)

#df_by_area['Postcode Area'].duplicated().sum() no Postcode Area duplicates

In [ ]:
# To find out how dense the fan base in each area is, we normalise

df_by_area['Qty Per 1K Pop'] = round(df_by_area['Qty']
                                     / df_by_area['pop_area']*1000, 2)
df_by_area['Total Rev Per 1K Pop'] = round(df_by_area['Total Revenue'] / df_by_area['pop_area']*1000, 2)

df_by_area['Pop Area'] = round(df_by_area['pop_area'] / 1000)
df_by_area['Pop Area'] = df_by_area[['Pop Area']].applymap('{:.0f}K'.format)['Pop Area']

df_by_area = df_by_area.sort_values(by = 'Qty Per 1K Pop', ascending =False)

In [ ]:
df_by_area.head()

### General Tickets

In [ ]:
df_by_area[(df_by_area['Ticket Type'] == 'General')][
['pcd_area',
 #'Ticket Type',
 'Postcode Area',
 'Pop Area',
 #'pop_area',
 'Qty',
 'Qty Per 1K Pop',
 'Total Revenue',
 #'Latitude',
 #'Longitude',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,87)).drop(columns = 'index').head(15) #.reindex(list(range(1,12)))

### Season Tickets

In [ ]:
df_by_area[(df_by_area['Ticket Type'] == 'Season') & (df_by_area['pcd_area'] == 'NR')]

In [ ]:
df_by_area[df_by_area['Ticket Type'] == 'Season'][
['pcd_area',
 #'Ticket Type',
 'Postcode Area',
 'Pop Area',
 #'pop_area',
 'Qty',
 #'Latitude',
 #'Longitude',
 'Qty Per 1K Pop',
 'Total Revenue',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,20)).drop(columns = 'index').head(12) #.reindex(list(range(1,12)))

In [ ]:
tickets[tickets['Ticket Type'] == 'General']['Total Revenue'].sum() /  tickets[tickets['Ticket Type'] == 'Season']['Total Revenue'].sum()

In [ ]:
tickets[tickets['Ticket Type'] == 'General']['Qty'].sum() /  tickets[tickets['Ticket Type'] == 'Season']['Qty'].sum()

## Group by District

In [171]:
df.groupby(['pcd_dist', 'Ticket Type'], as_index=False).agg({ #
    'Postcode Area': 'first', 
    'Qty': 'count',
    'Total Revenue': 'sum', 
    'pop_dist': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
}).sort_values(by = 'Total Revenue', ascending =False).head(5)

,pcd_dist,Ticket Type,Postcode Area,Qty,Total Revenue,pop_dist,Latitude,Longitude
0,AL1,General,St Albans,2989,"33,481.35","39,039.00",51.75,-0.32
8,AL4,General,St Albans,2043,"20,484.21","28,431.00",51.76,-0.30
6,AL3,General,St Albans,1349,"14,273.99","29,625.00",51.76,-0.35
1,AL1,Season,St Albans,142,"13,651.00","39,039.00",51.75,-0.32
9,AL4,Season,St Albans,120,"9,703.00","28,431.00",51.76,-0.30


In [173]:
df.groupby(['pcd_dist', 'Ticket Type'], as_index=False).agg({ #
    'Postcode Area': 'first', 
    'Qty': 'sum',
    'Total Revenue': 'sum', 
    'pop_dist': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
}).sort_values(by = 'Total Revenue', ascending =False).head(5)

,pcd_dist,Ticket Type,Postcode Area,Qty,Total Revenue,pop_dist,Latitude,Longitude
0,AL1,General,St Albans,4182,"33,481.35","39,039.00",51.75,-0.32
8,AL4,General,St Albans,2802,"20,484.21","28,431.00",51.76,-0.30
6,AL3,General,St Albans,1843,"14,273.99","29,625.00",51.76,-0.35
1,AL1,Season,St Albans,154,"13,651.00","39,039.00",51.75,-0.32
9,AL4,Season,St Albans,147,"9,703.00","28,431.00",51.76,-0.30


In [ ]:
df_by_district = df.groupby(['pcd_dist', 'Ticket Type'], as_index=False).agg({ #
    'Postcode Area': 'first', 
    'Qty': 'sum',
    'Total Revenue': 'sum', 
    'pop_dist': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
}).sort_values(by = 'Total Revenue', ascending =False)

df_by_district.head(2)

#df_by_district['Postcode Area'].duplicated().sum() no Postcode Area duplicates

In [ ]:
df_by_district['Qty Per 1K Pop'] = round(df_by_district['Qty'] / df_by_district['pop_dist']*1000, 2)
df_by_district['Total Rev Per 1K Pop'] = round(df_by_district['Total Revenue'] / df_by_district['pop_dist']*1000, 2)

df_by_district['Pop Dist'] = round(df_by_district['pop_dist'] / 1000)

df_by_district['Pop Dist'] = df_by_district[['Pop Dist']].applymap('{:.0f}K'.format)['Pop Dist']

df_by_district.sort_values(by = 'Qty Per 1K Pop', ascending =False).head(15)

### General Tickets

In [ ]:
df_by_district[df_by_district['Ticket Type'] == 'General'][
['pcd_dist',
 #'Ticket Type',
 'Postcode Area',
 'Pop Dist',
 'Qty',
 'Qty Per 1K Pop',
 'Total Revenue',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,573)).drop(columns = 'index').head(35) #.reindex(list(range(1,12)))

### Season Tickets

In [ ]:
df_by_district[df_by_district['Ticket Type'] == 'Season'][
['pcd_dist',
 #'Ticket Type',
 'Postcode Area',
 'Pop Dist',
 'Qty',
 'Qty Per 1K Pop',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Total Revenue',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,61)).drop(columns = 'index').head(20) #.reindex(list(range(1,12)))

## Group by Sector

In [ ]:
df_by_sector = df.groupby(['pcd_sect', 'Ticket Type'], as_index=False).agg({ #
    'Postcode Area': 'first', 
    'Qty': 'sum',
    'Total Revenue': 'sum', 
    'pop_sect': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
}).sort_values(by = 'Total Revenue', ascending =False)

df_by_sector.head(5)

#df_by_sector['Postcode Area'].duplicated().sum() no Postcode Area duplicates

In [ ]:
df_by_sector['Qty Per 1K Pop'] = round(df_by_sector['Qty'] / df_by_sector['pop_sect']*1000, 2)
df_by_sector['Total Rev Per 1K Pop'] = round(df_by_sector['Total Revenue'] / df_by_sector['pop_sect']*1000, 2)

df_by_sector['Pop Sec'] = round(df_by_sector['pop_sect'] / 1000)

df_by_sector['Pop Sec'] = df_by_sector[['Pop Sec']].applymap('{:.0f}K'.format)['Pop Sec']

df_by_sector.sort_values(by = 'Qty Per 1K Pop', ascending =False).head(5)

### General Tickets

In [ ]:
df_by_sector[df_by_sector['Ticket Type'] == 'General'][
['pcd_sect',
 #'Ticket Type',
 'Postcode Area',
 'Pop Sec',
 'Qty',
 'Qty Per 1K Pop',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Total Revenue',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,934)).drop(columns = 'index').head(35) #.reindex(list(range(1,12)))

### Season Tickets

In [ ]:
df_by_sector[df_by_sector['Ticket Type'] == 'Season'][
['pcd_sect',
 #'Ticket Type',
 'Postcode Area',
 'Pop Sec',
 'Qty',
 'Qty Per 1K Pop',
 'Total Revenue',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,95)).drop(columns = 'index').head(30) #.reindex(list(range(1,12)))

# Cut off

## Area

In [ ]:
df_by_area[df_by_area['Ticket Type'] == 'General'].sort_values(by = 'Qty Per 1K Pop', ascending =False).head(15)

In [ ]:
# median of General
df_by_area[
(df_by_area['pcd_area'] != 'AL') & 
(df_by_area['Ticket Type'] == 'General')]['Qty Per 1K Pop'].median()

In [ ]:
# median of Season
df_by_area[
(df_by_area['pcd_area'] != 'AL') & 
(df_by_area['Ticket Type'] == 'Season')]['Qty Per 1K Pop'].median()

In [ ]:
sns.boxplot(data= df_by_area[#(df_by_area['Ticket Type'] == 'General') & 
            (df_by_area['pcd_area'] != 'AL')],  x = 'Qty Per 1K Pop', y = 'Ticket Type').axvline(0.17, color=".3", dashes=(2, 2));

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 2))


sns.boxplot(data= df_by_area[(df_by_area['Ticket Type'] == 'Season') & 
            (df_by_area['pcd_area'] != 'AL')],  x = 'Qty Per 1K Pop', y = 'Ticket Type', width = 0.4).axvline(0.03, color=".3", dashes=(2, 2));

plt.show()

In [ ]:
# Histogram General

df_by_area[
(df_by_area['pcd_area'] != 'AL') & (df_by_area['Ticket Type'] == 'General')
].plot(y = 'Qty Per 1K Pop', kind = 'hist'); 

In [ ]:
# Total Rev Boxplot
sns.boxplot(data= df_by_area[df_by_area['pcd_area'] != 'AL'],  x = 'Total Rev Per 1K Pop', y = 'Ticket Type').axvline(2, color=".3", dashes=(2, 2));

In [ ]:
# Histogram Season

df_by_area[
(df_by_area['pcd_area'] != 'AL') & (df_by_area['Ticket Type'] == 'Season')
].plot(y = 'Qty Per 1K Pop', kind = 'hist'); 

## District

In [ ]:
df_by_district[df_by_district['Ticket Type'] == 'General'][
['pcd_dist',
 #'Ticket Type',
 'Postcode Area',
 'Qty',
 'Qty Per 1K Pop',
 'Total Revenue',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,573)).drop(columns = 'index').head(20) #.reindex(list(range(1,12)))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))

sns.boxplot(data= df_by_district[(df_by_district['Ticket Type'] == 'General') & 
            (df_by_district['pcd_dist'] != 'AL')],  x = 'Qty Per 1K Pop', y = 'Ticket Type').axvline(0.8, color=".3", dashes=(2, 2));

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))

sns.boxplot(data= df_by_district[(df_by_district['Ticket Type'] == 'General') & 
            (df_by_district['pcd_dist'] != 'AL')],  x = 'Total Rev Per 1K Pop', y = 'Ticket Type').axvline(0.8, color=".3", dashes=(2, 2));

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))


sns.boxplot(data= df_by_district[(df_by_district['Ticket Type'] == 'Season') & 
            (df_by_district['pcd_dist'] != 'AL')],  x = 'Qty Per 1K Pop', y = 'Ticket Type', width = 0.4).axvline(0.03, color=".3", dashes=(2, 2));

plt.show()

## Sector

In [ ]:
df_by_sector[df_by_sector['Ticket Type'] == 'General'][
['pcd_sect',
 #'Ticket Type',
 'Postcode Area',
 'Qty',
 'Qty Per 1K Pop',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Total Revenue',
 'Total Rev Per 1K Pop']
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,970)).drop(columns = 'index').head(15) #.reindex(list(range(1,12)))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))

sns.boxplot(data= df_by_sector[(df_by_sector['Ticket Type'] == 'General') & 
            (df_by_sector['pcd_sect'] != 'AL')],  x = 'Qty Per 1K Pop', y = 'Ticket Type').axvline(0.8, color=".3", dashes=(2, 2));

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))


sns.boxplot(data= df_by_sector[(df_by_sector['Ticket Type'] == 'Season') & 
            (df_by_sector['pcd_sect'] != 'AL')],  x = 'Qty Per 1K Pop', y = 'Ticket Type', width = 0.4).axvline(0.15, color=".3", dashes=(2, 2));

plt.show()

# Delete unnecessary dfs

##### https://stackoverflow.com/questions/57505460/why-does-jupyter-notebook-suddenly-slow-down-so-much

#### 1. See all of the loaded data frames loaded in this notebook

In [ ]:
%whos DataFrame

In [ ]:
lst = [tickets]
del tickets # dfs still in list
del lst     # memory release now

In [ ]:
%whos GeoDataFrame

#### 2. Delete

In [ ]:
lst = [#general_tickets,
       #pcdf,
       #popdf,
       #population,
       postcode_area_names,
       postcode_areas,
       #season_tickets,
       #tickets,
       #tickets_pcd,
       ukpostcodes]
del population, postcode_area_names, postcode_areas, ukpostcodes # dfs still in list
del lst     # memory release now

#### 3. Current CPU and RAM usage

In [ ]:
#!/usr/bin/env python
import psutil
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory()
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

# Maps 2024

## By Area (Qty % ~ with labels)

### Data Prep

In [ ]:
######### BOUNDARY DATA

gdf_area_boundary = gpd.read_file("dataset/opendoorlogistics.com/Areas.shp")

gdf_area_boundary.to_crs("EPSG:4326", inplace=True)

In [ ]:
# print(gdf.crs)      # This should show the CRS of the shapefile data

In [ ]:
gdf_area_boundary.head(2)

In [ ]:
df_by_area.head(2)

In [ ]:
######### JOINED

gdf_area = gdf_area_boundary.merge(df_by_area, how = 'left', left_on='name', right_on='pcd_area')

In [ ]:
######## REMOVE ST ALBANS

gdf_area = gdf_area[(gdf_area['name'] != 'AL')]

In [ ]:
######## GENERAL & SEASON TICKETS

gdf_area_season = gdf_area[gdf_area['Ticket Type'] == 'Season']

gdf_area_general = gdf_area[gdf_area['Ticket Type'] == 'General']

In [175]:
######## LOG SCALE (to see small nuances)

gdf_area['log Qty Per 1K Pop'] = np.log1p(gdf_area['Qty Per 1K Pop'])

NameError: name 'gdf_area' is not defined

In [ ]:
######## LIMIT TO TOP 

# gdf_area = gdf_area.sort_values(by = 'Qty Per 1K Pop', ascending = False).head(15)

#### For postcode areas ['SY', 'LD', 'NP', 'TF', 'DY', 'TR', 'WN', 'BL', 'OL', 'WF', 'HG']

#### We have not made a single sale

#### I do have the boundary data but

In [ ]:
gdf_area_boundary[gdf_area_boundary['name'].isin(['SY', 'LD', 'NP', 'TF', 'DY', 'TR', 'WN', 'BL', 'OL', 'WF', 'HG']) == True ].head(2)

In [ ]:
gdf_area[gdf_area['name'].isin(['SY', 'LD', 'NP', 'TF', 'DY', 'TR', 'WN', 'BL', 'OL', 'WF', 'HG']) == True ].head(2)

In [ ]:
gdf_area[gdf_area['pcd_area'].isin(['SY', 'LD', 'NP', 'TF', 'DY', 'TR', 'WN', 'BL', 'OL', 'WF', 'HG']) == True ]

In [ ]:
df[df['pcd_area'].isin(['SY', 'LD', 'NP', 'TF', 'DY', 'TR', 'WN', 'BL', 'OL', 'WF', 'HG']) == True ]

In [ ]:
df[df['pcd_area'] == 'SY']

In [ ]:
tickets[tickets['Outcode'].str.startswith('SY') == True ].head(2)

# tickets.head(2)

### General Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_area_general,
    geojson=gdf_area_general.geometry.__geo_interface__,
    locations=gdf_area_general.index,
    color="Qty Per 1K Pop", # color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.5,
    width=1100, 
    height=800
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 8
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_area_general.geometry.centroid.y,
    lon=gdf_area_general.geometry.centroid.x,
    mode='text',
    text=gdf_area_general['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'Qty Per 1K Pop'
)
)

fig.show();

### Season Tickets

In [ ]:
df_by_area[df_by_area['pcd_area'] == 'B']

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_area_season,
    geojson=gdf_area_season.geometry.__geo_interface__,
    locations=gdf_area_season.index,
    color="Qty Per 1K Pop", # color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.5,
    width=1100, 
    height=800
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 8
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_area_season.geometry.centroid.y,
    lon=gdf_area_season.geometry.centroid.x,
    mode='text',
    text=gdf_area_season['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'Qty Per 1K Pop'
)
)

fig.show()

## By District (Qty % ~ with labels)

### Data Prep

In [ ]:
######### BOUNDARY DATA

gdf_district_boundary = gpd.read_file("dataset/opendoorlogistics.com/Districts.shp")

gdf_district_boundary.to_crs("EPSG:4326", inplace=True)

In [ ]:
gdf_district_boundary.head(2)

In [ ]:
df_by_district.head(2)

In [ ]:
######### JOINED

gdf_district = gdf_district_boundary.merge(df_by_district, how = 'left', left_on='name', right_on='pcd_dist')


In [ ]:
######## REMOVE ST ALBANS

#gdf_district = gdf_district[(gdf_district['name'].str.startswith('AL') == False)]

In [ ]:
######## LOG SCALE (to see small nuances)

gdf_district['log Qty Per 1K Pop'] = np.log1p(gdf_district['Qty Per 1K Pop'])    

In [ ]:
######## GENERAL & SEASON TICKETS

gdf_district_season = gdf_district[gdf_district['Ticket Type'] == 'Season']

gdf_district_general = gdf_district[gdf_district['Ticket Type'] == 'General']

In [ ]:
######## LIMIT OUTPUT

#gdf_boundary = gdf_boundary[(gdf_boundary['Latitude'] < 52.78) & (gdf_boundary['Longitude'] > -1.63)] 

#gdf_district = gdf_district[(gdf_district['Latitude'] < 52.78) & (gdf_district['Longitude'] > -6)] 

### General Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_district_general,
    geojson=gdf_district_general.geometry.__geo_interface__,
    locations=gdf_district_general.index,
    color="Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=600
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_district_general.geometry.centroid.y,
    lon=gdf_district_general.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_district_general['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'Qty Per 1K Pop'
)
)

fig.show()

### Season Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_district_season,
    geojson=gdf_district_season.geometry.__geo_interface__,
    locations=gdf_district_season.index,
    color="Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=600
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_district_season.geometry.centroid.y,
    lon=gdf_district_season.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_district_season['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'Qty Per 1K Pop'
)
)

fig.show()

### Log Scale: General Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_district_general,
    geojson=gdf_district_general.geometry.__geo_interface__,
    locations=gdf_district_general.index,
    color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=800
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_district_general.geometry.centroid.y,
    lon=gdf_district_general.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_district_general['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'log Qty Per 1K Pop'
)
)

fig.show()

### Log Scale: Season Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_district_season,
    geojson=gdf_district_season.geometry.__geo_interface__,
    locations=gdf_district_season.index,
    color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=600
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_district_season.geometry.centroid.y,
    lon=gdf_district_season.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_district_season['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'log Qty Per 1K Pop'
)
)

fig.show()

## By Sector (Qty % ~ with labels)

### Data Prep

In [ ]:
######### BOUNDARY DATA

gdf_sector_boundary = gpd.read_file("dataset/opendoorlogistics.com/Sectors.shp")

gdf_sector_boundary.to_crs("EPSG:4326", inplace=True)

In [ ]:
gdf_sector_boundary.head(2)

In [ ]:
gdf_sector_boundary['name'][0]

In [ ]:
gdf_sector_boundary[gdf_sector_boundary['name'].str.startswith('AL4 ')]

In [ ]:
######### JOINED

gdf_sector = gdf_sector_boundary.merge(df_by_sector, how = 'left', left_on='name', right_on='pcd_sect')


In [ ]:
######## LOG SCALE (to see small nuances)

gdf_sector['log Qty Per 1K Pop'] = np.log1p(gdf_sector['Qty Per 1K Pop'])    

In [ ]:
######## GENERAL & SEASON TICKETS

gdf_sector_season = gdf_sector[gdf_sector['Ticket Type'] == 'Season']

gdf_sector_general = gdf_sector[gdf_sector['Ticket Type'] == 'General']

### General Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_sector_general,
    geojson=gdf_sector_general.geometry.__geo_interface__,
    locations=gdf_sector_general.index,
    color="Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=600
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_sector_general.geometry.centroid.y,
    lon=gdf_sector_general.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_sector_general['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'Qty Per 1K Pop'
)
)

fig.show()

### Season Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_sector_season,
    geojson=gdf_sector_season.geometry.__geo_interface__,
    locations=gdf_sector_season.index,
    color="Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=600
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_sector_season.geometry.centroid.y,
    lon=gdf_sector_season.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_sector_season['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'Qty Per 1K Pop'
)
)

fig.show()

### Log Scale: General Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_sector_general,
    geojson=gdf_sector_general.geometry.__geo_interface__,
    locations=gdf_sector_general.index,
    color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=800
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_sector_general.geometry.centroid.y,
    lon=gdf_sector_general.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_sector_general['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'log Qty Per 1K Pop'
)
)

fig.show()

### Log Scale: Season Tickets

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_sector_season,
    geojson=gdf_sector_season.geometry.__geo_interface__,
    locations=gdf_sector_season.index,
    color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=800
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_sector_season.geometry.centroid.y,
    lon=gdf_sector_season.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_sector_season['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'log Qty Per 1K Pop'
)
)

fig.show()

# General 2024 & General 2023

---
## General Tickets 2024 ~ U12 & U18 by sector

### Under 12 - 2024

In [ ]:
#df[(df['Ticket Type'] == 'General') & df['Type'].isin(['Under 12'])]

In [ ]:
general_under12_2024 = df[(df['Ticket Type'] == 'General') & df['Type'].isin(['Under 12'])].groupby(['pcd_sect', 'Type'], as_index=False).agg({ #'Next Gen 18-23',
    'Postcode Area': 'first', 
    'Qty': 'sum',
    'Total Revenue': 'sum', 
    'pop_sect': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
}).sort_values(by = 'Total Revenue', ascending =False)

In [ ]:
general_under12_2024 = general_under12_2024[general_under12_2024['pop_sect'].isna() == False]

In [ ]:
general_under12_2024['Qty Per 1K Pop'] = round(general_under12_2024['Qty'] / general_under12_2024['pop_sect']*1000, 2).astype('int')

general_under12_2024['Total Rev Per 1K Pop'] = round(general_under12_2024['Total Revenue'] / general_under12_2024['pop_sect']*1000, 2)

general_under12_2024['Pop Sec'] = round(general_under12_2024['pop_sect'] / 1000)
general_under12_2024['Pop Sec'] = general_under12_2024[['Pop Sec']].applymap('{:.0f}K'.format)['Pop Sec']

general_under12_2024.sort_values(by = 'Qty Per 1K Pop', ascending =False).head()

In [ ]:
# Under 12 by Sector

general_under12_2024[
[#'Type',
 'pcd_sect', 
 'Postcode Area',
    'Pop Sec',
 'Qty',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Qty Per 1K Pop',
 #'Total Revenue',
 #'Total Rev Per 1K Pop'
]
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,165)).drop(columns = 'index').head(20) #.reindex(list(range(1,12)))

### Under 18 - 2024

In [ ]:
general_under18_2024 = df[(df['Ticket Type'] == 'General') & df['Type'].isin(['Under 18'])].groupby(['pcd_sect', 'Type'], as_index=False).agg({ #'Next Gen 18-23',
    'Postcode Area': 'first', 
    'Qty': 'sum',
    #'Total Revenue': 'sum', 
    'pop_sect': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
})

In [ ]:
general_under18_2024['Qty Per 1K Pop'] = round(general_under18_2024['Qty'] / general_under18_2024['pop_sect']*1000, 2).astype('int')

#general_under18_2024['Total Rev Per 1K Pop'] = round(general_under18_2024['Total Revenue'] / general_under18_2024['pop_sect']*1000, 2).astype('int')

general_under18_2024['Pop Sec'] = round(general_under18_2024['pop_sect'] / 1000)
general_under18_2024['Pop Sec'] = general_under18_2024[['Pop Sec']].applymap('{:.0f}K'.format)['Pop Sec']

general_under18_2024.sort_values(by = 'Qty Per 1K Pop', ascending =False).head(2)

In [ ]:
# Under 18 by Sector

general_under18_2024[
['pcd_sect',
 'Postcode Area',
 'Type',
 'Pop Sec',
 'Qty',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Qty Per 1K Pop',
 #'Total Revenue',
 #'Total Rev Per 1K Pop'
]
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,237)).drop(columns = 'index').head(35) #.reindex(list(range(1,12)))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))

sns.boxplot(data= general_under18_2024,  x = 'Qty Per 1K Pop').axvline(0.8, color=".3", dashes=(2, 2));

plt.show()

## Map Plots: General 2024

### Under 12 ~ Log Scale

#### Data Prep

In [ ]:
######### BOUNDARY DATA

gdf_sector_boundary = gpd.read_file("dataset/opendoorlogistics.com/Sectors.shp")

gdf_sector_boundary.to_crs("EPSG:4326", inplace=True)

In [ ]:
gdf_sector_boundary.head(2)

In [ ]:
general_under12_2024

In [ ]:
general_under12_2024 = general_under12_2024[general_under12_2024['pop_sect'] != 'AL5 9']

In [ ]:
######### JOINED

gdf_sector_general_u12_2024 = gdf_sector_boundary.merge(general_under12_2024, how = 'left', left_on='name', right_on='pcd_sect')


In [ ]:
gdf_sector_general_u12_2024[gdf_sector_general_u12_2024['Postcode Area'].str.startswith('St') == True].head(2)

In [ ]:
######## LOG SCALE (to see small nuances)

gdf_sector_general_u12_2024['log Qty Per 1K Pop'] = np.log1p(gdf_sector_general_u12_2024['Qty Per 1K Pop'])    

In [ ]:
gdf_sector_general_u12_2024.to_crs("EPSG:4326", inplace=True)

#### Log Scale

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_sector_general_u12_2024,
    geojson=gdf_sector_general_u12_2024.geometry.__geo_interface__,
    locations=gdf_sector_general_u12_2024.index,
    color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['log Qty Per 1K Pop'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=600
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_sector_general_u12_2024.geometry.centroid.y,
    lon=gdf_sector_general_u12_2024.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_sector_general_u12_2024['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'log Qty Per 1K Pop'
)
)

fig.show()

### Under 18 ~ Log Scale

#### Data Prep

In [ ]:
gdf_sector_boundary.head(2)

In [ ]:
general_under18_2024.head(2)

In [ ]:
######### JOINED

gdf_sector_general_u18_2024 = gdf_sector_boundary.merge(general_under18_2024, how = 'left', left_on='name', right_on='pcd_sect')


In [ ]:
######## LOG SCALE (to see small nuances)

gdf_sector_general_u18_2024['log Qty Per 1K Pop'] = np.log1p(gdf_sector_general_u18_2024['Qty Per 1K Pop'])    

#### Log Scale

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_sector_general_u18_2024,
    geojson=gdf_sector_general_u18_2024.geometry.__geo_interface__,
    locations=gdf_sector_general_u18_2024.index,
    color="log Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['Qty'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=600
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_sector_general_u18_2024.geometry.centroid.y,
    lon=gdf_sector_general_u18_2024.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_sector_general_u18_2024['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'log Qty Per 1K Pop'
)
)

fig.show()

---
## General Tickets 2023

## Read data

In [ ]:
general_tickets_2023 = pd.read_excel('dataset/general_tickets_2023.xlsx')

In [ ]:
general_tickets_2023.head()

In [ ]:
general_tickets_2023.info()

## df_2023 ~ Tickets and Postcodes

### Ticket Prices

In [ ]:
general_tickets_2023.head(2)

In [ ]:
#df.groupby('Type').agg({
#    'Price' : 'mean'
#})

In [ ]:
ticket_prices = pd.DataFrame({'Type':['Adult', 'Carer','Concession', 'Next Gen 18-23', 'Under 18', 'Under 12', 'Hospitality', 'Carer'],
                             'Price':[10, 0, 8, 6, 4, 0, 50, 0]})

In [ ]:
ticket_prices

In [ ]:
ticket_prices2 = pd.DataFrame({'Type':['Adult', 'Carer','Concession', 'Concession - CH STH', 'Concession - PL STH', 'Next Gen 18-23', 'Under 18', 'Under 12', 'Hospitality', 'Carer'],
                             'Price':[16.5, 0, 11, 11, 11, 8, 6, 0, 50, 0]})

ticket_prices2.head(12)

In [ ]:
general_tickets_2023_try = general_tickets_2023.merge(
    ticket_prices2,
    how = 'left')

In [ ]:
general_tickets_2023_try['Total Revenue'] = general_tickets_2023_try['Price']*general_tickets_2023_try['Qty']

In [ ]:
general_tickets_2023_try.head()

In [ ]:
general_tickets_2023_try['Total Revenue'].sum()

### Merged

In [ ]:
df_2023 = general_tickets_2023.merge(
    pcdf[[
        'Postcode', 
        'Latitude', 
        'Longitude', 
        'pcd_sect', 
        'pcd_dist', 
        'pcd_area', 
        'Postcode Area']],
    how = 'left', 
    left_on = 'Postcode', 
    right_on = 'Postcode').merge(
    popdf[[
        'Postcode',
        'pop_area',
        'pop_dist',
        'pop_sect']],
    how = 'left',
    left_on = 'Postcode',
    right_on = 'Postcode')#.merge(
    #ticket_prices,
    #how = 'left')


In [ ]:
#df_2023['Total Revenue'] = df_2023['Price']*df_2023['Qty']


df_2023 = df_2023[['id',
                   'order_item_id',
                   'Fixture',
                   'Type',
                   #'Price',
                   'Qty',
                   #'Total Revenue',
                   'Postcode',
                   'Latitude',
                   'Longitude',
                   'pcd_sect',
                   #'pcd_dist',
                   #'pcd_area',
                   'Postcode Area',
                   #'pop_area',
                   #'pop_dist',
                   'pop_sect']]

In [ ]:
df_2023.head(2)

---
## General Tickets 2023 ~ U12 & U18 by sector

### Under 12 - 2023

In [ ]:
general_under12_2023 = df_2023[df_2023['Type'].isin(['Under 12'])].groupby(['pcd_sect', 'Type'], as_index=False).agg({ #'Next Gen 18-23',
    'Postcode Area': 'first', 
    #'Total Revenue': 'sum',
    'Qty': 'sum',
    'pop_sect': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
})

In [ ]:
general_under12_2023['Qty Per 1K Pop'] = round(general_under12_2023['Qty'] / general_under12_2023['pop_sect']*1000, 2).astype('int')

#general_under12_2023['Total Rev Per 1K Pop'] = round(general_under12_2023['Total Revenue'] / general_under12_2023['pop_sect']*1000, 2)

general_under12_2023['Pop Sec'] = round(general_under12_2023['pop_sect'] / 1000)
general_under12_2023['Pop Sec'] = general_under12_2023[['Pop Sec']].applymap('{:.0f}K'.format)['Pop Sec']

general_under12_2023.sort_values(by = 'Qty Per 1K Pop', ascending =False).head()

In [ ]:
# Under 12 by Sector

general_under12_2023[
[#'Type',
 'pcd_sect',
 'Postcode Area',
    'Pop Sec',
 'Qty',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Qty Per 1K Pop',
# 'Total Revenue',
# 'Total Rev Per 1K Pop'
]
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,137)).drop(columns = 'index').head() #.reindex(list(range(1,12)))

### Under 18 - 2023

In [ ]:
general_under18_2023 = df_2023[df_2023['Type'].isin(['Under 18'])].groupby(['pcd_sect', 'Type'], as_index=False).agg({ #'Next Gen 18-23',
    'Postcode Area': 'first', 
    'Qty': 'sum',
    #'Total Revenue': 'sum',
    'pop_sect': 'first',
    'Latitude' : 'mean',
    'Longitude' : 'mean'
})

In [ ]:
#general_under18_2023[general_under18_2023['pop_sect'].isna()]

In [ ]:
general_under18_2023 = general_under18_2023[general_under18_2023['pop_sect'].isna() == False]

In [ ]:
general_under18_2023['Qty Per 1K Pop'] = round(general_under18_2023['Qty'] / general_under18_2023['pop_sect']*1000, 2).astype('int')

#general_under18_2023['Total Rev Per 1K Pop'] = round(general_under18_2023['Total Revenue'] / general_under18_2023['pop_sect']*1000, 2)

general_under18_2023['Pop Sec'] = round(general_under18_2023['pop_sect'] / 1000)
general_under18_2023['Pop Sec'] = general_under18_2023[['Pop Sec']].applymap('{:.0f}K'.format)['Pop Sec']

general_under18_2023.sort_values(by = 'Qty Per 1K Pop', ascending = False).head()

In [ ]:
# Under 18 by Sector

general_under18_2023[
['pcd_sect',
 'Type',
 'Postcode Area',
 'Pop Sec',
 'Qty',
 #'pop_area',
 #'Latitude',
 #'Longitude',
 'Qty Per 1K Pop',
#'Total Revenue',
 #'Total Rev Per 1K Pop'
]
].sort_values(by = 'Qty Per 1K Pop', ascending =False
             ).reset_index().set_axis(range(1,209)).drop(columns = 'index').head(20) #.reindex(list(range(1,12)))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))

sns.boxplot(data= general_under18_2023,  x = 'Qty Per 1K Pop').axvline(0.8, color=".3", dashes=(2, 2));

plt.show()

---
# Year on Year Analysis

## Under 12

In [ ]:
general_under12_2023_2024 = general_under12_2024[
[
    'pcd_sect',
    'Type',
    'Postcode Area',
    #'Total Revenue',
    'pop_sect',
    'Pop Sec',
    'Latitude',
    'Longitude',
    'Qty',
    'Qty Per 1K Pop',
    #'Total Rev Per 1K Pop'
]
].merge(general_under12_2023[
        [
            'pcd_sect',
            #'Type',
            #'Postcode Area',
            'Qty',
            #'Total Revenue',
            #'pop_sect',
            #'Latitude',
            #'Longitude',
            'Qty Per 1K Pop',
            #'Total Rev Per 1K Pop'
        ]
        ],
        how = 'left',
        on = 'pcd_sect',
        suffixes = ('_24', '_23'))#.sort_values(by = 'Qty Per 1K Pop', ascending = False)

In [ ]:
general_under12_2023_2024['YoY Qty Per 1K Pop'] = (general_under12_2023_2024['Qty_24'] - general_under12_2023_2024['Qty_23'])/general_under12_2023_2024['pop_sect']*1000

In [ ]:
general_under12_2023_2024['Qty_23'] = general_under12_2023_2024['Qty_23'].fillna(0).astype('int')

general_under12_2023_2024['Qty Per 1K Pop_23'] = general_under12_2023_2024['Qty Per 1K Pop_23'].fillna(0).astype('int')

#general_under12_2023_2024.rename(columns=
#    {'Qty Per 1K Pop_23' : 'Qty_23 Per 1K Pop',
#     'Qty Per 1K Pop_24' : 'Qty_24 Per 1K Pop'},
#    inplace = True
#)

In [ ]:
general_under12_2023_2024['YoY Qty'] = general_under12_2023_2024['Qty_24'] - general_under12_2023_2024['Qty_23']

In [ ]:
general_under12_2023_2024.head()

In [ ]:
#general_under12_2023_2024[['Qty_24',	
#                           'Qty Per 1K Pop_24',	
#                           'Qty_23',	
#                           'Qty Per 1K Pop_23',
#                           'YoY Qty',
#                          'YoY Qty Per 1K Pop']] = general_under12_2023_2024[['Qty_24',	
#                                                                              'Qty Per 1K Pop_24',	
#                                                                              'Qty_23',	
#                                                                              'Qty Per 1K Pop_23',
#                                                                              'YoY Qty',
#                                                                              'YoY Qty Per 1K Pop']].round().astype('Int64')

In [ ]:
general_under12_2023_2024[
['pcd_sect',
 'Type',
 'Postcode Area',
 'Pop Sec',
 'Latitude',
 'Longitude',
 'Qty_23',
 'Qty_24',
 'Qty Per 1K Pop_23',
 'Qty Per 1K Pop_24',
 'YoY Qty',
 'YoY Qty Per 1K Pop']
].sort_values(by = 'YoY Qty Per 1K Pop', ascending = False).head(15)

In [ ]:
general_under12_2023_2024[
['pcd_sect',
 'Type',
 'Postcode Area',
 'Pop Sec',
 #'Latitude',
 #'Longitude',
 #'Qty_23',
 #'Qty_24',
 #'Qty Per 1K Pop_23',
 #'Qty Per 1K Pop_24',
 'YoY Qty',
 'YoY Qty Per 1K Pop']
].sort_values(by = 'YoY Qty Per 1K Pop', ascending = False).reset_index().set_axis(range(1,165)).drop(columns = 'index').head(35)

## YoY Under 12 Map

### Data Prep

In [ ]:
######### BOUNDARY DATA

#gdf_sector_boundary = gpd.read_file("dataset/opendoorlogistics.com/Sectors.shp")

#gdf_sector_boundary.to_crs("EPSG:4326", inplace=True)

In [ ]:
gdf_sector_boundary.head(2)

In [ ]:
#gdf_sector_boundary[gdf_sector_boundary['name'].isin(['AL1 9', 'AL5 9']) == True]

In [ ]:
######### JOINED

gdf_yoy_u12 = gdf_sector_boundary[gdf_sector_boundary['name'].isin(['AL1 9', 'AL5 9']) == False].merge(general_under12_2023_2024, how = 'left', left_on='name', right_on='pcd_sect')


In [ ]:
gdf_yoy_u12[gdf_yoy_u12['Postcode Area'].str.startswith('St') == True].head(2)

In [ ]:
######## LOG SCALE (to see small nuances)

gdf_yoy_u12['log YoY Qty Per 1K Pop'] = np.log1p(gdf_yoy_u12['YoY Qty Per 1K Pop'])    

In [ ]:
gdf_yoy_u12.to_crs("EPSG:4326", inplace=True)

### YoY, Under 12 Plot

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_yoy_u12,
    geojson=gdf_yoy_u12.geometry.__geo_interface__,
    locations=gdf_yoy_u12.index,
    color="YoY Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['YoY Qty Per 1K Pop'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=800
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_yoy_u12.geometry.centroid.y,
    lon=gdf_yoy_u12.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_yoy_u12['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'YoY Qty Per 1K Pop'
)
)

fig.show()

## Under 18

In [ ]:
general_under18_2023_2024 = general_under18_2024[
[
    'pcd_sect',
    'Type',
    'Postcode Area',
    #'Total Revenue',
    'pop_sect',
    'Pop Sec',
    'Latitude',
    'Longitude',
    'Qty',
    'Qty Per 1K Pop',
    #'Total Rev Per 1K Pop'
]
].merge(general_under18_2023[
        [
            'pcd_sect',
            #'Type',
            #'Postcode Area',
            'Qty',
            #'Total Revenue',
            #'pop_sect',
            #'Latitude',
            #'Longitude',
            'Qty Per 1K Pop',
            #'Total Rev Per 1K Pop'
        ]
        ],
        how = 'left',
        on = 'pcd_sect',
        suffixes = ('_24', '_23'))#.sort_values(by = 'Qty Per 1K Pop', ascending = False)

In [ ]:
general_under18_2023_2024['YoY Qty Per 1K Pop'] = (
    general_under18_2023_2024['Qty_24'] - general_under18_2023_2024['Qty_23']
)/general_under18_2023_2024['pop_sect']*1000

In [ ]:
#general_under18_2023_2024['YoY Qty Per 10K Pop'] = (
#    general_under18_2023_2024['Qty_24'] - general_under18_2023_2024['Qty_23']
#)/general_under18_2023_2024['pop_sect']*10000

In [ ]:
general_under18_2023_2024['Qty_23'] = general_under18_2023_2024['Qty_23'].fillna(0).astype('int')

general_under18_2023_2024['Qty Per 1K Pop_23'] = general_under18_2023_2024['Qty Per 1K Pop_23'].fillna(0).astype('int')

#general_under18_2023_2024.rename(columns=
#    {'Qty Per 1K Pop_23' : 'Qty_23 Per 1K Pop',
#     'Qty Per 1K Pop_24' : 'Qty_24 Per 1K Pop'},
#    inplace = True
#)

In [ ]:
general_under18_2023_2024['YoY Qty'] = general_under18_2023_2024['Qty_24'] - general_under18_2023_2024['Qty_23']

#general_under18_2023_2024['YoY Qty Per 1K Pop'] = general_under18_2023_2024['Qty Per 1K Pop_24'] - general_under18_2023_2024['Qty Per 1K Pop_23']

In [ ]:
general_under18_2023_2024.head()

In [ ]:
#general_under18_2023_2024[['Qty_24',	
#                           'Qty Per 1K Pop_24',	
#                           'Qty_23',	
#                           'Qty Per 1K Pop_23',
#                           'YoY Qty',
#                          'YoY Qty Per 1K Pop']] = general_under18_2023_2024[['Qty_24',	
#                                                                              'Qty Per 1K Pop_24',	
#                                                                              'Qty_23',	
#                                                                              'Qty Per 1K Pop_23',
#                                                                              'YoY Qty',
#                                                                              'YoY Qty Per 1K Pop']].round().astype('Int64')

In [ ]:
general_under18_2023_2024[
['pcd_sect',
 'Type',
 'Postcode Area',
 'Pop Sec',
 'Latitude',
 'Longitude',
 'Qty_23',
 'Qty_24',
 'YoY Qty',
 'Qty Per 1K Pop_23',
 'Qty Per 1K Pop_24',
 'YoY Qty Per 1K Pop']
].sort_values(by = 'YoY Qty Per 1K Pop', ascending = False).head(15)

In [ ]:
general_under18_2023_2024[
['pcd_sect',
 'Type',
 'Postcode Area',
 'Pop Sec',
 #'Latitude',
 #'Longitude',
 #'Qty_23',
 #'Qty_24',
 'YoY Qty',
 #'Qty Per 1K Pop_23',
 #'Qty Per 1K Pop_24',
 'YoY Qty Per 1K Pop',
 #'YoY Qty Per 10K Pop'
]
].sort_values(by = 'YoY Qty Per 1K Pop', ascending = False).reset_index().set_axis(range(1,237)).drop(columns = 'index').head(15)

## YoY Under 18 Map

### Data Prep

In [ ]:
######### BOUNDARY DATA

#gdf_sector_boundary = gpd.read_file("dataset/opendoorlogistics.com/Sectors.shp")

#gdf_sector_boundary.to_crs("EPSG:4326", inplace=True)

In [ ]:
gdf_sector_boundary.head(2)

In [ ]:
#gdf_sector_boundary[gdf_sector_boundary['name'].isin(['AL1 9', 'AL5 9']) == True]

In [ ]:
######### JOINED

gdf_yoy_u18 = gdf_sector_boundary[gdf_sector_boundary['name'].isin(['AL1 9', 'AL5 9']) == False].merge(general_under18_2023_2024, how = 'left', left_on='name', right_on='pcd_sect')


In [ ]:
gdf_yoy_u18[gdf_yoy_u18['Postcode Area'].str.startswith('St') == True].head(2)

In [ ]:
######## LOG SCALE (to see small nuances)

gdf_yoy_u18['log YoY Qty Per 1K Pop'] = np.log1p(gdf_yoy_u18['YoY Qty Per 1K Pop'])    

In [ ]:
gdf_yoy_u18.to_crs("EPSG:4326", inplace=True)

### YoY, Under 18 Map

In [ ]:
######## PLOT USING PLOTLY

#mapboxtoken="pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGtsYnQ0MGk4eDJ2cXdlem1ibmw5eiJ9.PhskQoeb0KroCq2sF1EEeQ"
mapboxtoken = 'pk.eyJ1IjoiZWxpZmRhdGEiLCJhIjoiY20yNGpxazY4MGV3ZzJpczZwMm4xd3BhOSJ9.lAY-XV-pV_b90V3O7jvqVw'
mapboxstyle="mapbox://styles/elifdata/cm24l3c3g009501pihjqxaaik"

fig = px.choropleth_mapbox(
    gdf_yoy_u18,
    geojson=gdf_yoy_u18.geometry.__geo_interface__,
    locations=gdf_yoy_u18.index,
    color="YoY Qty Per 1K Pop",
    hover_name = 'name', # ['name', 'Longitude', 'Latitude'],
    hover_data = ['YoY Qty Per 1K Pop'],
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.6,
    width=1100, 
    height=800
).update_layout(
    mapbox={
        "accesstoken":mapboxtoken,
        "style": mapboxstyle,
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 10
    },
    margin={"l":0,"r":0,"t":0,"b":0}
).add_trace(
    go.Scattermapbox(
    lat=gdf_yoy_u18.geometry.centroid.y,
    lon=gdf_yoy_u18.geometry.centroid.x,  # Longitude from your data
    mode='text',
    text=gdf_yoy_u18['name'],  # The 'name' column as labels
    textfont=dict(size=12, color='black'),  # Text size and color
    textposition='middle center',  # Position of the label
    name = 'YoY Qty Per 1K Pop'
)
)

fig.show()


# Revenue 2023 ~ General Tickets only

In [ ]:
pd.DataFrame({'Ticket Type': 'General', 'Total Revenue': df_2023['Total Revenue'].sum()}, index=[0])

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 1.25))

sns.barplot(x = 'Total Revenue',
            y = 'Ticket Type',
            data = pd.DataFrame({'Ticket Type': 'General', 'Total Revenue': df_2023['Total Revenue'].sum()}, 
                                index=[0]),
            width=0.35)

plt.title('Total Revenue in 2023 ',fontsize=10)
plt.xticks(np.arange(0, 240001, step=50000))
ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel(' ')
plt.ylabel(' ')

In [ ]:
# General Tickets Purchases 2023
plt.figure(figsize=(4, 2.5))

# Bar plot using seaborn
sns.barplot(x='Qty',
            y='Type',
            color='darkblue',
            data=df_2023 # & (df['Price'] > 0)
            [['Qty', 'Type']].groupby('Type', as_index = False).sum('Qty').sort_values('Qty', ascending=False))

# Adding labels and title
plt.xlabel(' ')
plt.ylabel(' ')
plt.title('General Tickets Purchases 2023') 

# Show plot

plt.show()

In [ ]:
# Total Revenue General
plt.figure(figsize=(4, 2.5))

# Bar plot using seaborn
sns.barplot(x='Total Revenue',
            y='Type',
            color='darkblue',
            data=df_2023[['Total Revenue', 'Type']].groupby('Type', as_index = False).sum('Total Revenue').sort_values('Total Revenue', ascending=False))

# Adding labels and title
plt.xlabel(' ')
plt.ylabel(' ')
plt.title('Total Revenue in 2023 (General)') 

# Show plot
plt.show()


# Revenue 2024

### General tickets brought 3 times more revenue

In [1]:
df.groupby('Ticket Type', as_index=False).agg({
    'Total Revenue': 'sum',
    'Qty': 'sum'
}).sort_values('Qty', ascending=False)#.head(10)

#125177.44/44229.50

NameError: name 'df' is not defined

In [ ]:
import matplotlib as mpl

fig, ax = plt.subplots(figsize=(4.5, 2))

sns.barplot(x = 'Total Revenue',
            y = 'Ticket Type',
            data = df.groupby('Ticket Type')['Total Revenue'].sum().reset_index(name="Total Revenue").sort_values(by = 'Total Revenue', 
                                                                                            ascending = False).head(5)
           )

plt.title('Total Revenue in 2024 ',fontsize=10)
plt.xticks(np.arange(0, 150001, step=50000))
ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'));
plt.xlabel(' ')
plt.ylabel(' ')


plt.show()

In [ ]:
df.head(1)

In [ ]:
df[['Qty', 'Type', 'Price', 'Ticket Type', 'Total Revenue']].groupby(['Ticket Type', 'Type'], as_index = True).agg(
    Sum_Qty =pd.NamedAgg(column="Qty", aggfunc="sum"),
    #Avg_Rev=pd.NamedAgg(column="Total Revenue", aggfunc="mean"),
    Sum_Revenue =pd.NamedAgg(column="Total Revenue", aggfunc="sum"))#.sort_values('Avg_Price', ascending=False)

#43*68

In [ ]:
df_rev = df[['Qty', 'Type', 'Price', 'Ticket Type', 'Total Revenue']].groupby(['Ticket Type', 'Type'], as_index = False).agg(
    Sum_Qty =pd.NamedAgg(column="Qty", aggfunc="sum"),
    #Avg_Rev=pd.NamedAgg(column="Total Revenue", aggfunc="mean"),
    Sum_Revenue =pd.NamedAgg(column="Total Revenue", aggfunc="sum"))#.sort_values('Avg_Price', ascending=False)

#43*68

df_rev[df_rev['Ticket Type'] == 'General'].sort_values('Sum_Revenue', ascending=False)

In [ ]:
# General Tickets Purchases
plt.figure(figsize=(4, 2.5))

# Bar plot using seaborn
sns.barplot(x='Qty',
            y='Type',
            color='darkblue',
            data=df[(df['Ticket Type'] == 'General') # & (df['Price'] > 0)
            ][['Qty', 'Type']].groupby('Type', as_index = False).sum('Qty').sort_values('Qty', ascending=False))

# Adding labels and title
plt.xlabel(' ')
plt.ylabel(' ')
plt.title('General Tickets Purchases 2024') 

# Show plot

plt.show()

In [ ]:
# Season Tickets Purchases
plt.figure(figsize=(4, 2.5))

# Bar plot using seaborn
sns.barplot(x='Qty',
            y='Type',
            color='darkblue',
            data=df[df['Ticket Type'] == 'Season'][['Qty', 'Type']].groupby('Type', as_index = False).sum('Qty').sort_values('Qty', ascending=False))

# Adding labels and title
plt.xlabel(' ')
plt.ylabel(' ')
plt.title('Season Tickets Purchases 2024') 

# Show plot
plt.show()

In [ ]:
# Total Revenue General
plt.figure(figsize=(4, 2.5))

# Bar plot using seaborn
sns.barplot(x='Total Revenue',
            y='Type',
            color='darkblue',
            data=df[df['Ticket Type'] == 'General'][['Total Revenue', 'Type']].groupby('Type', as_index = False).sum('Total Revenue').sort_values('Total Revenue', ascending=False))

# Adding labels and title
plt.xlabel(' ')
plt.ylabel(' ')
plt.title('Total Revenue in 2024 (General)') 

# Show plot
plt.show()

In [ ]:
# Total Revenue Season
plt.figure(figsize=(4, 2.5))

# Bar plot using seaborn
sns.barplot(x='Total Revenue',
            y='Type',
            color='darkblue',
            data=df[df['Ticket Type'] == 'Season'][['Total Revenue', 'Type']].groupby('Type', as_index = False).sum('Total Revenue').sort_values('Total Revenue', ascending=False))

# Adding labels and title
plt.xlabel(' ')
plt.ylabel(' ')
plt.title('Total Revenue in 2024 (Season)') 

# Show plot
plt.show()

### Bar Charts

In [ ]:
df['Total Revenue'].sum()

In [ ]:
df[df['pcd_area'] == 'AL']['Total Revenue'].sum() / df['Total Revenue'].sum() *100

In [ ]:
df[df['pcd_area'] == 'HP']['Total Revenue'].sum() / df['Total Revenue'].sum() *100

In [ ]:
df[df['pcd_area'] == 'WD']['Total Revenue'].sum() / df['Total Revenue'].sum() *100

In [ ]:
df_by_area[['Postcode Area', 'Total Revenue']].groupby('Postcode Area').sum('Total Revenue').sort_values('Total Revenue', ascending=False).head()

In [ ]:
plt.figure(figsize=(6, 4))  # You can adjust the size to your preference

# Bar plot using seaborn
sns.barplot(x='Total Revenue',
            y='Postcode Area',
            color='darkblue',
            data=df_by_area.head(10))

# Adding labels and title
plt.xlabel(' ')
plt.ylabel(' ')
plt.title('Total Revenue by Postcode Areas')

# Show plot
plt.show()

In [ ]:
general_tickets_2023.head()

# Delete Maps

 ### density_mapbox (Col: Qty)

In [ ]:
# get UK boundaries
resUK = requests.get(
    "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Planning_Authorities_April_2023_Boundaries_UK_BUC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
)


# scatter the cities and add layer that shows municiple boundary
px.density_mapbox(df[df['Price'] > 0], lat='Latitude', lon='Longitude', z='Qty', 
                  radius=30,
                  center=dict(lat=51.75, lon=-0.31), 
                  zoom=0,
                  mapbox_style="open-street-map",
                  opacity=0.5,
                  title='Nr of Tickets (Excluding £0)',
                  width=1000, height=1000).update_layout(
    mapbox={
        "style": "carto-positron",
        "zoom": 12,
        "layers": [
            {
                "source": resUK.json(),
                "type": "line",
                "color": "green",
                "line": {"width": 1},
            }
        ],
    }
)

### choropleth_mapbox (Sum: Qty)

In [ ]:
######### BOUNDARY DATA


# Fetch the GeoJSON data (https://geoportal.statistics.gov.uk/datasets/ons::local-authority-districts-may-2024-boundaries-uk-buc-2/explore)

url = "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Authority_Districts_May_2024_Boundaries_UK_BUC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

response_boundary = requests.get(url) # get() sends an HTTP GET request to the specified URL, and returns a Response object. (# requests is the library)


dict_boundary = response_boundary.json() # Convert the JSON response object (which contains feature collections) to a dictionary

                                         # FeatureCollection = An object defining a layer of features whose geometry and attributes will be stored directly within the web map

# Convert the data to a GeoDataFrame

gdf_boundary = gpd.GeoDataFrame.from_features(dict_boundary['features'], crs="EPSG:4326")




######### OUR DATA AS GEO-DATA-FRAME

# Add a column named 'geomery' in the format of point long lat

gdf_df = df.copy()

gdf_df['geometry'] = gdf_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

gdf_df = gpd.GeoDataFrame(gdf_df, geometry='geometry', crs="EPSG:4326")

#gdf_df.head()



######### JOINED

# Spatial join to sum quantities within each boundary
joined = gdf_boundary.sjoin(gdf_df,how='left', predicate='contains') ## joined = gdf_df.sjoin(gdf_boundary,how='left', predicate='within')


# Aggregate quantities based on boundaries
quantity_sum = joined.groupby(joined.index)['Qty'].sum()

gdf_boundary['Qty'] = gdf_boundary.index.map(quantity_sum).fillna(0)

# Plot using Plotly
fig = px.choropleth_mapbox(
    gdf_boundary,
    geojson=gdf_boundary.geometry.__geo_interface__,
    locations=gdf_boundary.index,
    color="Qty",
    hover_name = 'LAD24NM',
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.5
).update_layout(
    mapbox={
        "style": "carto-positron",
        "center": {"lon": -0.31, "lat": 51.75},
        "zoom": 8
    },
    margin={"l":0,"r":0,"t":0,"b":0}
)

fig.show()


### Manually Adjust the Color Scale

In [ ]:
###################fig = px.choropleth_mapbox(
###################    gdf_boundary,
###################    geojson=gdf_boundary.geometry.__geo_interface__,
###################    locations=gdf_boundary.index,
###################    color='Qty',
###################    hover_name = 'LAD24NM',
###################    hover_data = ['LAD24CD', 'LAD24NM', 'LAD24NMW', 'LONG', 'LAT', 'Qty'],
###################    color_continuous_scale="inferno",
###################    range_color=(0, 1000),  # Set the range to focus on the majority of data
###################    opacity=0.75
###################).update_layout(
###################    mapbox={
###################        "style": "carto-positron",
###################        "center": {"lon": -0.31, "lat": 51.86},
###################        "zoom": 8.2
###################    },
###################    margin={"l":0,"r":0,"t":0,"b":0}
###################)
###################
###################fig.show()
###################

### All Wards

In [ ]:
# Westminister Parliamentary Wards

# https://martinjc.github.io/UK-GeoJSON/



######### BOUNDARY

# Fetch boundary data from web
gdf_boundary_UKwards = gpd.read_file('https://martinjc.github.io/UK-GeoJSON/json/eng/topo_wards.json',
                              usecols = ['id', 'geometry']).set_crs("EPSG:4326")


######### OUR DATA AS GEO-DATA-FRAME

# Add a column named 'geomery' in the format of point long lat

gdf_df = df.copy()

gdf_df['geometry'] = gdf_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

gdf_df = gpd.GeoDataFrame(gdf_df, geometry='geometry', crs="EPSG:4326")



######### joined_UKwards

# Spatial join to sum quantities within each boundary
joined_UKwards = gdf_boundary_UKwards.sjoin(gdf_df,how='left', predicate='contains') ## joined_UKwards = gdf_df.sjoin(gdf_boundary_UKwards,how='left', predicate='within')


# Aggregate quantities based on boundaries
quantity_sum_UKwards = joined_UKwards.groupby(joined_UKwards.index)['Qty'].sum()

gdf_boundary_UKwards['Qty'] = gdf_boundary_UKwards.index.map(quantity_sum_UKwards).fillna(0)

# Plot using Plotly
fig = px.choropleth_mapbox(
    gdf_boundary_UKwards,
    geojson=gdf_boundary_UKwards.geometry.__geo_interface__,
    locations=gdf_boundary_UKwards.index,
    color="Qty",
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.5
).update_layout(
    mapbox={
        "style": "carto-positron",
        "center": {"lon": -0.31, "lat": 51.85},
        "zoom": 8.25
    },
    margin={"l":0,"r":0,"t":0,"b":0}
)

fig.show()


### Supporter population by Ward (in St Albans only)

In [ ]:
# Westminister Parliamentary Wards

# https://martinjc.github.io/UK-GeoJSON/



######### BOUNDARY

# Fetch boundary data from web
gdf_boundary_stalbans_wards = gpd.read_file("https://martinjc.github.io/UK-GeoJSON/json/eng/wards_by_lad/topo_E07000240.json",
                              usecols = ['id', 'geometry']).set_crs("EPSG:4326")


######### OUR DATA AS GEO-DATA-FRAME

# Add a column named 'geomery' in the format of point long lat

gdf_df = df.copy()

gdf_df['geometry'] = gdf_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

gdf_df = gpd.GeoDataFrame(gdf_df, geometry='geometry', crs="EPSG:4326")



######### JOINED

# Spatial join to sum quantities within each boundary
joined = gdf_boundary_stalbans_wards.sjoin(gdf_df,how='left', predicate='contains') ## joined = gdf_df.sjoin(gdf_boundary_wards,how='left', predicate='within')


# Aggregate quantities based on boundaries
quantity_sum_wards = joined.groupby(joined.index)['Qty'].sum()

gdf_boundary_stalbans_wards['Qty'] = gdf_boundary_stalbans_wards.index.map(quantity_sum_wards).fillna(0)

# Plot using Plotly
fig = px.choropleth_mapbox(
    gdf_boundary_stalbans_wards,
    geojson=gdf_boundary_stalbans_wards.geometry.__geo_interface__,
    locations=gdf_boundary_stalbans_wards.index,
    color="Qty",
    color_continuous_scale="Viridis",  # Use Viridis color scale for better visualization
    opacity=0.75
).update_layout(
    mapbox={
        "style": "carto-positron",
        "center": {"lon": -0.31, "lat": 51.77},
        "zoom": 10.4
    },
    margin={"l":0,"r":0,"t":0,"b":0}
)

fig.show()


---
### Facetted USA map

In [ ]:
import plotly.graph_objects as go
import pandas as pd
df_facet = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/1962_2006_walmart_store_openings.csv')
df_facet.head()

data = []
layout = dict(
    title = 'New Walmart Stores per year 1962-2006<br>\
Source: <a href="http://www.econ.umn.edu/~holmes/data/WalMart/index.html">\
University of Minnesota</a>',
    # showlegend = False,
    autosize = False,
    width = 1000,
    height = 900,
    hovermode = False,
    legend = dict(
        x=0.7,
        y=-0.1,
        bgcolor="rgba(255, 255, 255, 0)",
        font = dict( size=11 ),
    )
)
years = df_facet['YEAR'].unique()

for i in range(len(years)):
    geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
    lons = list(df_facet[ df_facet['YEAR'] == years[i] ]['LON'])
    lats = list(df_facet[ df_facet['YEAR'] == years[i] ]['LAT'])
    # Walmart store data
    data.append(
        dict(
            type = 'scattergeo',
            showlegend=False,
            lon = lons,
            lat = lats,
            geo = geo_key,
            name = int(years[i]),
            marker = dict(
                color = "rgb(0, 0, 255)",
                opacity = 0.5
            )
        )
    )
    # Year markers
    data.append(
        dict(
            type = 'scattergeo',
            showlegend = False,
            lon = [-78],
            lat = [47],
            geo = geo_key,
            text = [years[i]],
            mode = 'text',
        )
    )
    layout[geo_key] = dict(
        scope = 'usa',
        showland = True,
        landcolor = 'rgb(229, 229, 229)',
        showcountries = False,
        domain = dict( x = [], y = [] ),
        subunitcolor = "rgb(255, 255, 255)",
    )


def draw_sparkline( domain, lataxis, lonaxis ):
    ''' Returns a sparkline layout object for geo coordinates  '''
    return dict(
        showland = False,
        showframe = False,
        showcountries = False,
        showcoastlines = False,
        domain = domain,
        lataxis = lataxis,
        lonaxis = lonaxis,
        bgcolor = 'rgba(255,200,200,0.0)'
    )

# Stores per year sparkline
layout['geo44'] = draw_sparkline({'x':[0.6,0.8], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 30.0]}, {'range':[0.0, 40.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df_facet.groupby(by=['YEAR']).count()['storenum']/1e1),
        lon = list(range(len(df_facet.groupby(by=['YEAR']).count()['storenum']/1e1))),
        line = dict( color = "rgb(0, 0, 255)" ),
        name = "New stores per year<br>Peak of 178 stores per year in 1990",
        geo = 'geo44',
    )
)

# Cumulative sum sparkline
layout['geo45'] = draw_sparkline({'x':[0.8,1], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 50.0]}, {'range':[0.0, 50.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df_facet.groupby(by=['YEAR']).count().cumsum()['storenum']/1e2),
        lon = list(range(len(df_facet.groupby(by=['YEAR']).count()['storenum']/1e1))),
        line = dict( color = "rgb(214, 39, 40)" ),
        name ="Cumulative sum<br>3176 stores total in 2006",
        geo = 'geo45',
    )
)

z = 0
COLS = 5
ROWS = 9
for y in reversed(range(ROWS)):
    for x in range(COLS):
        geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
        layout[geo_key]['domain']['x'] = [float(x)/float(COLS), float(x+1)/float(COLS)]
        layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), float(y+1)/float(ROWS)]
        z=z+1
        if z > 42:
            break

fig = go.Figure(data=data, layout=layout)
fig.update_layout(width=800)
fig.show()